In [54]:
import datetime
import pandas as pd

In [55]:

latest_datetime_wue_ex = "2021-07-10T16:00:00"
# change the string to datetime object
latest_datetime_wue_ex = datetime.datetime.strptime(latest_datetime_wue_ex, '%Y-%m-%dT%H:%M:%S')
print(latest_datetime_wue_ex)

first_date = latest_datetime_wue_ex + datetime.timedelta(hours=1)
last_date = latest_datetime_wue_ex + datetime.timedelta(hours=24)
print(first_date)
print(last_date)


2021-07-10 16:00:00
2021-07-10 17:00:00
2021-07-11 16:00:00


In [56]:
# # get just the date
# first_date_day = first_date.date()
# last_date_day = last_date.date()
# print(first_date_day)
# print(last_date_day)
# # to string
# first_date_day_str = first_date_day.strftime('%Y-%m-%d')
# last_date_day_str = last_date_day.strftime('%Y-%m-%d')
# print("string")
# print(first_date_day_str)
# print(last_date_day_str)

In [57]:
import requests
import io

In [58]:
# 1. Set up the API key and the base URL
api_key = 'QPX3CJ476JEHWLUYT3TC559U8'
    
# Define the endpoint URL and parameters
base_url = 'https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/w%C3%BCrzburg'
params = {
    'unitGroup': 'metric',  # Use 'us' for Fahrenheit, 'metric' for Celsius
    'include': 'hours',  # Include hourly weather data
    'key': api_key,
    'contentType': 'csv'  # Request CSV format
}
# Construct the full API URL
x = first_date.date().strftime('%Y-%m-%d')
y = last_date.date().strftime('%Y-%m-%d')
url = f"{base_url}/{x}/{y}"

# Send the GET request
response = requests.get(url, params=params)

if response.status_code == 200:
    
    # Read the response as a DataFrame
    weather_data_24h = pd.read_csv(io.StringIO(response.text))
    
    # dropping the columns that we don't need later
    
else:
    raise Exception(f"Error in get_weather_forecast_24h: {response.text}")

# convert the csv data to a pandas dataframe
weather_data_24h_df = pd.DataFrame(weather_data_24h)

print(weather_data_24h.shape)
# print min und max datetime
print(weather_data_24h['datetime'].min())
print(weather_data_24h['datetime'].max())
weather_data_24h.head()



(48, 24)
2021-07-10T00:00:00
2021-07-11T23:00:00


,name,datetime,temp,feelslike,dew,humidity,precip,precipprob,preciptype,snow,...,sealevelpressure,cloudcover,visibility,solarradiation,solarenergy,uvindex,severerisk,conditions,icon,stations
0,"WÃ¼rzburg, Bayern, Deutschland",2021-07-10T00:00:00,14.8,14.8,14.3,96.80,0.0,0,NaN,0,...,1022.0,52.0,17.0,0,0.0,0,NaN,Partially cloudy,partly-cloudy-night,"00191,05705,05149,06336,03257,02600"
1,"WÃ¼rzburg, Bayern, Deutschland",2021-07-10T01:00:00,13.8,13.8,13.2,95.71,0.0,0,NaN,0,...,1022.0,54.3,28.0,0,0.0,0,NaN,Partially cloudy,partly-cloudy-night,"00191,05705,05149,06336,03257,02600"
2,"WÃ¼rzburg, Bayern, Deutschland",2021-07-10T02:00:00,13.5,13.5,13.3,98.64,0.0,0,NaN,0,...,1022.0,13.0,0.0,1,0.0,0,NaN,Clear,clear-night,"00191,D4939,05705,05149,06336,03257,02600"
3,"WÃ¼rzburg, Bayern, Deutschland",2021-07-10T03:00:00,12.8,12.8,12.6,98.73,0.0,0,NaN,0,...,1021.3,11.5,0.0,1,0.0,0,NaN,Clear,clear-night,"00191,D4939,05705,05149,06336,03257,02600"
4,"WÃ¼rzburg, Bayern, Deutschland",2021-07-10T04:00:00,12.5,12.5,12.3,98.75,0.0,0,NaN,0,...,1021.0,47.8,2.0,1,0.0,0,NaN,Partially cloudy,partly-cloudy-night,"00191,D4939,05705,05149,06336,03257,02600"


In [59]:
#print(first_date) 2021-07-10 17:00:00
#print(last_date) 2021-07-11 16:00:00
# beides inklusive

# change first_date and last_date to string and add a T between date and time
first_date_str = first_date.strftime('%Y-%m-%d') + 'T' + first_date.strftime('%H:%M:%S')
last_date_str= last_date.strftime('%Y-%m-%dT%H:%M:%S')
print(first_date)
print(last_date)

# filter the data to get the data from the first_date to the last_date
weather_data_24h_df = weather_data_24h_df[(weather_data_24h_df['datetime'] >= first_date_str) & (weather_data_24h_df['datetime'] <= last_date_str)]

print(weather_data_24h_df.shape)


2021-07-10 17:00:00
2021-07-11 16:00:00
(24, 24)


# Plot pedestrian_count

In [81]:
import plotly.express as px

In [82]:
# get the data for the plot from the database

import sqlite3

# connect to the database
conn = sqlite3.connect('data.db')

# get the data from the database
wue_data = pd.read_sql('SELECT location_id, pedestrians_count, timestamp FROM data ORDER BY timestamp', conn)
wue_data['date'] = wue_data['timestamp'].str.extract(r'(\d{4}-\d{2}-\d{2})')
wue_data['datetime'] = wue_data['timestamp'].str.split('+').str[0]

conn.close()

In [83]:
wue_data.head()

,location_id,pedestrians_count,timestamp,date,datetime
0,135.0,1346,2024-01-01T00:00:00+01:00,2024-01-01,2024-01-01T00:00:00
1,470.0,388,2024-01-01T00:00:00+01:00,2024-01-01,2024-01-01T00:00:00
2,476.0,1197,2024-01-01T00:00:00+01:00,2024-01-01,2024-01-01T00:00:00
3,135.0,772,2024-01-01T01:00:00+01:00,2024-01-01,2024-01-01T01:00:00
4,470.0,161,2024-01-01T01:00:00+01:00,2024-01-01,2024-01-01T01:00:00


In [84]:
wue_data_agg = wue_data.groupby(['location_id', 'datetime']).agg({'pedestrians_count': 'sum'}).reset_index()
wue_data_agg = wue_data_agg.merge(wue_data[['location_id', 'datetime', 'date']], on=['location_id', 'datetime'], how='left')
wue_data_agg.sort_values(by=['datetime', 'location_id'], inplace=True)
wue_data_agg.tail(24)

,location_id,datetime,pedestrians_count,date
4639,135.0,2024-07-14T00:00:00,334,2024-07-14
9294,470.0,2024-07-14T00:00:00,125,2024-07-14
13954,476.0,2024-07-14T00:00:00,378,2024-07-14
4640,135.0,2024-07-14T01:00:00,238,2024-07-14
9295,470.0,2024-07-14T01:00:00,44,2024-07-14
13955,476.0,2024-07-14T01:00:00,211,2024-07-14
4641,135.0,2024-07-14T02:00:00,0,2024-07-14
9296,470.0,2024-07-14T02:00:00,0,2024-07-14
13956,476.0,2024-07-14T02:00:00,0,2024-07-14
4642,135.0,2024-07-14T03:00:00,0,2024-07-14


In [90]:
# create a line plotly express plot with the data for the last 3 days in the wue_data_agg dataframe
# get the last 3 days values from the wue_data_agg dataframe in the column date
last_3_days = wue_data_agg['date'].unique()[-3:]
# filter the data to get the data for the last 3 days
wue_data_agg_3_days = wue_data_agg[wue_data_agg['date'].isin(last_3_days)]
# create the plot
fig = px.line(wue_data_agg_3_days, x='datetime', y='pedestrians_count', color='location_id', title='Pedestrians count for the last 3 days')
# add a line to the plot for the sum data of the different locations in wue_data_agg_3_days
sum_data = wue_data_agg_3_days.groupby('datetime').agg({'pedestrians_count': 'sum'}).reset_index()
fig.add_scatter(x=sum_data['datetime'], y=sum_data['pedestrians_count'], mode='lines', name='Sum')
# show the plot

fig.show()


# last_3_days = wue_data_agg['date'].unique()[-3:]
# # filter the data to get the data for the last 3 days
# wue_data_agg_3_days = wue_data_agg[wue_data_agg['date'].isin(last_3_days)]
# # create the plot
# fig = px.line(wue_data_agg_3_days, x='date', y='pedestrians_count', color='location_id', title='Pedestrians count for the last 3 days')

# fig.show()

C:\Users\User\AppData\Roaming\Python\Python311\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



# recursive updating lags for the prediction

In [1]:
import pandas as pd
import numpy as np

# Define the DataFrame
data = {
    'lag_1': [300, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    'lag_2': [200, 300, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    'lag_3': [100, 200, 300, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    'lag_4': [50, 100, 200, 300, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
}

df = pd.DataFrame(data)

# Define the function to calculate the next value
def next_val(current_val):
    # Example calculation for the next value
    return current_val + 100

# Iterate over the rows
for i in range(len(df) - 1):
    # Calculate the new lag_1 value for the next row
    if pd.notna(df.iloc[i, 0]):
        new_lag_1 = next_val(df.iloc[i, 0])
        
        # Update the next row's lag_1 value
        df.iloc[i + 1, 0] = new_lag_1
        
        # Update the subsequent lags
        for j in range(1, 4):
            if i + 1 + j < len(df):
                df.iloc[i + 1 + j, j] = new_lag_1

print(df)


     lag_1   lag_2   lag_3   lag_4
0    300.0   200.0   100.0    50.0
1    400.0   300.0   200.0   100.0
2    500.0   400.0   300.0   200.0
3    600.0   500.0   400.0   300.0
4    700.0   600.0   500.0   400.0
5    800.0   700.0   600.0   500.0
6    900.0   800.0   700.0   600.0
7   1000.0   900.0   800.0   700.0
8   1100.0  1000.0   900.0   800.0
9   1200.0  1100.0  1000.0   900.0
10  1300.0  1200.0  1100.0  1000.0
11  1400.0  1300.0  1200.0  1100.0
12  1500.0  1400.0  1300.0  1200.0
13  1600.0  1500.0  1400.0  1300.0
14  1700.0  1600.0  1500.0  1400.0
15  1800.0  1700.0  1600.0  1500.0
16  1900.0  1800.0  1700.0  1600.0
17  2000.0  1900.0  1800.0  1700.0
18  2100.0  2000.0  1900.0  1800.0
19  2200.0  2100.0  2000.0  1900.0
20  2300.0  2200.0  2100.0  2000.0
21  2400.0  2300.0  2200.0  2100.0
22  2500.0  2400.0  2300.0  2200.0
23  2600.0  2500.0  2400.0  2300.0


In [2]:
import pandas as pd
import numpy as np

# Define the DataFrame
data = {
    'lag_1': [300, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    'lag_2': [200, 300, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    'lag_3': [100, 200, 300, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan],
    'lag_4': [50, 100, 200, 300, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, np.nan]
}

df = pd.DataFrame(data)

# Define the function to calculate the next value
def next_val(current_val):
    # Example calculation for the next value
    return current_val + 100

# Iterate over the rows
for i in range(len(df) - 1):
    # Calculate the new lag_1 value for the next row
    if pd.notna(df.at[i, 'lag_1']):
        new_lag_1 = next_val(df.at[i, 'lag_1'])
        
        # Update the next row's lag_1 value
        df.at[i + 1, 'lag_1'] = new_lag_1
        
        # Update the subsequent lags
        for j in range(1, 4):
            if i + 1 + j < len(df):
                df.at[i + 1 + j, f'lag_{j + 1}'] = new_lag_1

print(df)


     lag_1   lag_2   lag_3   lag_4
0    300.0   200.0   100.0    50.0
1    400.0   300.0   200.0   100.0
2    500.0   400.0   300.0   200.0
3    600.0   500.0   400.0   300.0
4    700.0   600.0   500.0   400.0
5    800.0   700.0   600.0   500.0
6    900.0   800.0   700.0   600.0
7   1000.0   900.0   800.0   700.0
8   1100.0  1000.0   900.0   800.0
9   1200.0  1100.0  1000.0   900.0
10  1300.0  1200.0  1100.0  1000.0
11  1400.0  1300.0  1200.0  1100.0
12  1500.0  1400.0  1300.0  1200.0
13  1600.0  1500.0  1400.0  1300.0
14  1700.0  1600.0  1500.0  1400.0
15  1800.0  1700.0  1600.0  1500.0
16  1900.0  1800.0  1700.0  1600.0
17  2000.0  1900.0  1800.0  1700.0
18  2100.0  2000.0  1900.0  1800.0
19  2200.0  2100.0  2000.0  1900.0
20  2300.0  2200.0  2100.0  2000.0
21  2400.0  2300.0  2200.0  2100.0
22  2500.0  2400.0  2300.0  2200.0
23  2600.0  2500.0  2400.0  2300.0
